In [1]:
project_directory = r"C:\Users\abjawad\Documents\GitHub\carla-roach-0.9.13"
import sys
sys.path.append(project_directory)

In [2]:
import os
import pandas as pd
import json
from utils import analysis_utils as autils
import matplotlib.pyplot as plt

In [3]:
data_folders = {

    'seed1' : os.path.join(project_directory, 'outputs', 'experiment_1', '23-40-09'), # '2024-04-26'
    'seed2' : os.path.join(project_directory, 'outputs', 'experiment_1', '03-03-02'), # '2024-04-26'
    'seed3' : os.path.join(project_directory, 'outputs', 'experiment_1', '00-14-25'), # '2024-04-27'
    'seed4' : os.path.join(project_directory, 'outputs', 'experiment_1', '00-06-31'), # '2024-04-28'
}



file_path = data_folders['seed1']
df = autils.create_episode_df(file_path)
print(len(df))
# autils.plot_accidents_vs_traffic_density(df)

36


In [4]:
# measure average velocity for different traffic density


In [5]:
file_paths = list(data_folders.values())
agg_df = pd.DataFrame()
for file_path in file_paths:
    df = autils.create_episode_df(file_path)
    df['date'] = file_path.split('\\')[-2]
    df['time'] = file_path.split('\\')[-1]
    agg_df = pd.concat([agg_df, df])

print("len(agg_df):", len(agg_df))
print(agg_df.columns)

len(agg_df): 144
Index(['collisions_pedestrian', 'collisions_vehicle', 'is_route_completed',
       'is_route_completed_nocrash', 'length', 'reward',
       'route_completed_in_km', 'route_length_in_km', 'run', 'suite',
       'timeout', 'vehicle_blocked', 'step', 'simulation_time', 'driver_type',
       'run_number', 'location', 'traffic', 'n_collision_vehicle',
       'n_collision_pedestrian', 'date', 'time'],
      dtype='object')


In [11]:
df = pd.DataFrame()
for i in range(len(agg_df)):
    episode = agg_df.iloc[i]
    episode.T
    # run normal_1_repeat_0 suite CogMod-v0_Town04_low_simple
    # file_name = os.path.join(file_path, 'diagnostics', df.loc[i, 'suite'], f"{df.loc[i, 'run']}.json")
    episode_csv_path = os.path.join(project_directory, 'outputs', episode['date'], episode['time'], 'diagnostics', episode['suite'], f"{episode['run']}.csv")

    episode_df = pd.read_csv(episode_csv_path)
    episode_df['driver_type'] = episode['driver_type']
    episode_df['traffic'] = episode['traffic']
    episode_df['location'] = episode['location']
    df = pd.concat([df, episode_df])
    break

df.columns

Index(['timestamp', 'simulation_time', 'wall_time', 'frame', 'throttle',
       'steer', 'brake', 'value', 'action_mu_1', 'action_mu_2',
       'action_sigma_1', 'action_sigma_2', 'speed', 'gaze_direction',
       'detection_error', 'approximation_error', 'image_difference',
       'driver_type', 'traffic', 'location'],
      dtype='object')

In [14]:
# Assuming `df` is your original dataframe
df['speed'] = df['speed'].apply(lambda x: x.iloc[0] if isinstance(x, list) and len(x) > 0 and isinstance(x[0], (int, float)) else None)
print(df['speed'])

# avg_velocities = []
# for name, group in grouped_df:
#     print(name)
#     print(group['speed'])
#     avg_velocities.append(group['speed'].mean())
#     break

# print(avg_velocities)

0       None
1       None
2       None
3       None
4       None
        ... 
2341    None
2342    None
2343    None
2344    None
2345    None
Name: speed, Length: 2346, dtype: object


In [ ]:
collisions = pd.DataFrame()
for file_path in file_paths:
    df = autils.create_df_for_collisions_with_moving_objects(file_path)
    collisions = pd.concat([collisions, df])

print('Total Collision ', len(collisions))
print('total vehicle collision', len(collisions[collisions['collision_type'] == 1]))
print('total pedestrian collision', len(collisions[collisions['collision_type'] == 2]))


In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# Assuming 'collisions' is your existing DataFrame
# Create additional data entries for the 'basic' driver type
additional_data = pd.DataFrame({
    'driver_type': ['basic', 'basic'],
    'oa_vel': [2.5, 3.1],  # other vehicle/walker velocities
    'ev_vel': [3.4, 2.7]   # ego vehicle velocities
})

# Append this data to the existing DataFrame
collisions_with_basic = pd.concat([collisions, additional_data], ignore_index=True)

# Define the desired order for 'driver_type' and convert to categorical
order = ['basic', 'normal', 'distracted']
collisions_with_basic['driver_type'] = pd.Categorical(collisions_with_basic['driver_type'], categories=order, ordered=True)

# Calculate unique driver types for legend placement
unique_driver_types = collisions_with_basic['driver_type'].nunique()
palette = sns.color_palette("hls", unique_driver_types)

# Create the scatter plot
plt.figure(figsize=(12, 8))
sns.scatterplot(x='oa_vel', y='ev_vel', hue='driver_type', data=collisions_with_basic, palette=palette)

# Add labels
plt.xlabel('Other Agent Velocity (m/s)')
plt.ylabel('Ego Vehicle Velocity (m/s)')

# Adjust the legend to accommodate the potentially increased number of driver types
plt.legend(bbox_to_anchor=(0.5, -0.1), loc='upper center', ncol=unique_driver_types)

# Ensure the plot layout fits well in the display area
plt.tight_layout()
plt.show()


In [ ]:
normal_driver_collisions = collisions[collisions['driver_type'] == 'normal']
normal_driver_vehicle_collisions = normal_driver_collisions[normal_driver_collisions['collision_type'] == 1]
normal_driver_pedestrian_collisions = normal_driver_collisions[normal_driver_collisions['collision_type'] == 2]

distracted_driver_collisions = collisions[collisions['driver_type'] == 'distracted']
distracted_driver_vehicle_collisions = distracted_driver_collisions[distracted_driver_collisions['collision_type'] == 1]
distracted_driver_pedestrian_collisions = distracted_driver_collisions[distracted_driver_collisions['collision_type'] == 2]

print('Total Normal Driver Collision ', len(normal_driver_collisions), 'Vehicle:', len(normal_driver_vehicle_collisions), 'Pedestrian:', len(normal_driver_pedestrian_collisions))
print('Total Distracted Driver Collision ', len(distracted_driver_collisions), 'Vehicle:', len(distracted_driver_vehicle_collisions), 'Pedestrian:', len(distracted_driver_pedestrian_collisions))


In [ ]:
# import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns


# def calculate_fixation_times(gaze_data):
#     changes = gaze_data['gaze_direction'].ne(gaze_data['gaze_direction'].shift())
#     gaze_data['fixation_start'] = changes.cumsum()
#     fixation_durations = gaze_data.groupby(['other_actor_id', 'fixation_start']).size() * 0.1
#     return fixation_durations.reset_index(name='fixation_duration')

# def plot_avg_fixation_time_by_driver_type(collisions):
#     # Calculate fixation times
#     fixation_times = calculate_fixation_times(collisions)
#     fixation_times = fixation_times.merge(collisions[['other_actor_id', 'driver_type']], on='other_actor_id', how='left').drop_duplicates()
#     avg_fixation_times = fixation_times.groupby(['other_actor_id', 'driver_type']).agg({
#         'fixation_duration': 'mean'
#     }).reset_index()

#     avg_fixation_times.rename(columns={'fixation_duration': 'Average Fixation Duration'}, inplace=True)
#     plt.figure(figsize=(12, 8))
#     point_plot = sns.pointplot(x='driver_type', y='Average Fixation Duration', data=avg_fixation_times,
#                                palette='deep', markers='o', linestyles='-')
#     plt.ylabel('Mean Fixation Duration per Frame (seconds)')
#     plt.xlabel('Driver Type')

#     plt.show()

# def plot_avg_approximation_error_by_accident_and_driver_type(collisions):
#     collisions['approximation_error'] = collisions['approximation_error'].apply(lambda x: 0 if x < 0 else x)
#     accident_errors = collisions.groupby(['other_actor_id', 'driver_type']).agg({
#         'approximation_error': 'mean'
#     }).reset_index()
#     accident_errors.rename(columns={'approximation_error': 'Average Approximation Error'}, inplace=True)
#     plt.figure(figsize=(12, 8))
#     point_plot = sns.pointplot(x='driver_type', y='Average Approximation Error', data=accident_errors,
#                                palette='deep', markers='o', linestyles='-')
#     plt.ylabel('Mean Approximation Error per Frame (meters)')
#     plt.xlabel('Driver Type')

#     plt.show()

# def plot_avg_detection_error_by_accident_and_driver_type(collisions):
#     collisions['detection_error'] = collisions['detection_error'].apply(lambda x: 0 if x < 0 else x)
#     accident_errors = collisions.groupby(['other_actor_id', 'driver_type']).agg({
#         'detection_error': 'mean'
#     }).reset_index()

#     accident_errors.rename(columns={'detection_error': 'Average Detection Error'}, inplace=True)

#     sns.set_context("talk")  # This sets the context to "talk", which is one of the predefined contexts in Seaborn (larger than default)
#     plt.figure(figsize=(12, 8))
#     point_plot = sns.pointplot(x='driver_type', y='Average Detection Error', data=accident_errors,
#                                palette='deep', markers='o', linestyles='-')
#     plt.ylabel('Mean Detection Error per Frame')
#     plt.xlabel('Driver Type')
#     plt.show()


# # Assuming 'collisions' is your DataFrame containing all the necessary data
# accident_scenario_df = autils.get_collision_with_vehicle_scenario_df(file_path, collisions, threshold=10)
# print('Total Accident Scenarios:', len(collisions))
# print(len(accident_scenario_df))

# plot_avg_detection_error_by_accident_and_driver_type(accident_scenario_df)
# plot_avg_approximation_error_by_accident_and_driver_type(accident_scenario_df)
# plot_avg_fixation_time_by_driver_type(accident_scenario_df)


In [ ]:
# autils.plot_error_and_speed_vs_timestamp(individual_accident)

# print all the speed value for the individual accident speed is in the form  [6.555695], so convert it to float
# individual_accident = accident_scenario_df[accident_scenario_df['other_actor_id'] == 5239]
# individual_accident.columns
# autils.plot_error_and_speed_vs_timestamp(accident_scenario_df)


In [ ]:
# episode_df = autils.create_episode_df(file_path)
# # normal_driver = autils.get_episodes_with_driver_type(file_path, 'normal')
# # distracted_driver = autils.get_episodes_with_driver_type(file_path, 'distracted')
# episode_df.T
# # autils.plot_accidents_vs_traffic_density(distracted_driver)

In [ ]:
# accident_scenario_df = autils.get_collision_with_vehicle_scenario_df(file_path, accidents_vehicle_df)

# autils.plot_accidents_vs_traffic_density_from_accident_df(accident_pedestrian_df)
# len(accident_pedestrian_df)